# Get Starting

In [49]:
import pandas as pd
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')
df.head(2)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering The Data

In [50]:
df['Year'] = df['Year'].str[0:4]
df['Year'] = df['Year'].astype('int64')

later_than_2000 = df[df['Year'] > 2000]

award_categories = ['Actor -- Leading Role', 'Actor -- Supporting Role',
                   'Actress -- Leading Role', 'Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]
nominations.head(2)

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning Up Unnamed Columns

In [51]:
map_dict = {'YES': 1, 'NO': 0}
nominations.loc[:, 'Won'] = nominations.loc[:, 'Won?'].map(map_dict)
drop_c = ['Won?', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
         'Unnamed: 9', 'Unnamed: 10']
final_nominations = nominations.drop(drop_c, axis=1)
final_nominations.head(2)

/dataquest/system/env/python3/lib/python3.4/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/dataquest/system/env/python3/lib/python3.4/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0


## Cleaning Up The Additional Info Column

In [52]:
additional_info_one = final_nominations['Additional Info'].str.rstrip(" }'")
additional_info_two = additional_info_one.str.split(" {'")
final_nominations['Movie'] = additional_info_two.str[0]
final_nominations['Character'] = additional_info_two.str[1]
final_nominations.drop('Additional Info', axis=1)
final_nominations.head(2)

,Year,Category,Nominee,Additional Info,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0,True Grit,Rooster Cogburn


## Exporting To SQLite

In [54]:
import sqlite3
conn = sqlite3.connect('nominations.db')
final_nominations.to_sql('nominations', conn, index=False)

/dataquest/system/env/python3/lib/python3.4/site-packages/pandas/core/generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


## Verifying In SQL

In [56]:
print(conn.execute('PRAGMA table_info("nominations")').fetchall())
print(conn.execute('SELECT * FROM nominations LIMIT10').fetchall())
conn.close()

[(0, 'Year', 'INTEGER', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Nominee', 'TEXT', 0, None, 0), (3, 'Additional Info', 'TEXT', 0, None, 0), (4, 'Won', 'INTEGER', 0, None, 0), (5, 'Movie', 'TEXT', 0, None, 0), (6, 'Character', 'TEXT', 0, None, 0)]
[(2010, 'Actor -- Leading Role', 'Javier Bardem', "Biutiful {'Uxbal'}", 0, 'Biutiful', 'Uxbal'), (2010, 'Actor -- Leading Role', 'Jeff Bridges', "True Grit {'Rooster Cogburn'}", 0, 'True Grit', 'Rooster Cogburn'), (2010, 'Actor -- Leading Role', 'Jesse Eisenberg', "The Social Network {'Mark Zuckerberg'}", 0, 'The Social Network', 'Mark Zuckerberg'), (2010, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech {'King George VI'}", 1, "The King's Speech", 'King George VI'), (2010, 'Actor -- Leading Role', 'James Franco', "127 Hours {'Aron Ralston'}", 0, '127 Hours', 'Aron Ralston'), (2010, 'Actor -- Supporting Role', 'Christian Bale', "The Fighter {'Dicky Eklund'}", 1, 'The Fighter', 'Dicky Eklund'), (2010, 'Actor -- Supportin